<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER_Model_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [37]:
pip install sklearn-crfsuite


In [38]:
pip install nltk

In [0]:
import nltk

In [40]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
nltk.download('gazetteers')

[nltk_data] Downloading package gazetteers to /root/nltk_data...
[nltk_data]   Package gazetteers is already up-to-date!


True

In [42]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [0]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.stem.porter import *
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import names, ieer, gazetteers

In [0]:
locations = gazetteers.words()
locations = [x.lower() for x in locations]
names_list = names.words()
names_list = [x.lower() for x in names_list]

In [45]:
word = 'LA'
word.lower() in locations

True

In [46]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:


def file_to_list(file_name):  
  training_key_value = []
  word_dict = {}
  tags = {}
  word_list = []
  sentence = 1
  file1 = open(file_name, 'r')
  Lines = file1.readlines() 
  for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(([sentence, word, tagged[0][1], tag])) 
      tags.update(tagged)
    else:
      sentence = sentence + 1
    file1.close()
  return word_dict,word_list

#Train data
word_dict_train, word_list_train = file_to_list('train.txt')
# print(word_dict_train.keys())
# array_of_words_train = np.array(list(word_dict_train.keys()))
# array_of_values_train = np.array(list(word_dict_train.values()))
# X_Y_train = np.column_stack((array_of_words_train, array_of_values_train))

#Dev data
word_dict_dev,word_list_dev = file_to_list('dev.txt')
# array_of_words_dev = np.array(list(word_dict_dev.keys()))
# array_of_values_dev = np.array(list(word_dict_dev.values()))
# X_Y_dev = np.column_stack((array_of_words_dev, array_of_values_dev))




In [0]:
def file_to_list_test(file_name):  
  training_key_value = []
  word_dict = {}  
  word_list = []
  training_key_value = [] 
  test_word_list = []
  file1 = open(file_name, 'r')
  sentence_num = 1
  Lines = file1.readlines() 
  # print("print" + str(len(Lines[11].split('\n'))))
  for line in Lines:    
    if(len(line) > 1):      
      word = line.replace('\n', '')
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      # print(sentence_num)
      test_word_list.append(([sentence_num, word, tagged[0][1]]))
    else:
      # print("\nelse")
      sentence_num = sentence_num + 1
  file1.close()
  return test_word_list



#Test data
word_list_test = file_to_list_test('test_no_tag.txt')


In [49]:
train_data = pd.DataFrame(word_list_train)
print(train_data.head(n = 4))

dev_data = pd.DataFrame(word_list_dev)
print(dev_data.head(n = 4))


test_data = pd.DataFrame(word_list_test)
print(test_data.head(n = 4))
# print(word_list_test)

   0                1    2  3
0  1  @SammieLynnsMom   NN  O
1  1         @tg10781   NN  O
2  1             they  PRP  O
3  1             will   MD  O
   0       1   2  3
0  1    STOP  NN  O
1  1    WHAT  WP  O
2  1  YOU'RE  NN  O
3  1   DOING  NN  O
   0       1    2
0  1      We  PRP
1  1     're  VBP
2  1  slaves  NNS
3  1      to   TO


In [50]:
# train_data.columns = train_data.iloc[0]
# train_data = train_data[1:]
train_data.columns = ['Sentence','Word','POS','Tag']
# train_data = train_data.reset_index(drop=True)
#train_data.head(n=5)

# dev_data.columns = dev_data.iloc[0]
# dev_data = dev_data[1:]
dev_data.columns = ['Sentence','Word','POS','Tag']
# dev_data = dev_data.reset_index(drop=True)
#dev_data.head(n=5)

# test_data.columns = test_data.iloc[0]
# test_data = test_data[1:]
test_data.columns = ['Sentence','Word','POS']
# test_data = test_data.reset_index(drop=True)
test_data.head(n= 10)


,Sentence,Word,POS
0,1,We,PRP
1,1,'re,VBP
2,1,slaves,NNS
3,1,to,TO
4,1,a,DT
5,1,world,NN
6,1,we,PRP
7,1,mastered,VBN
8,1,.,.
9,2,I,PRP


In [51]:
test_data.shape

(42413, 3)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

class getsentence_ForTest(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p) for w, p in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist())
                                                      ]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [53]:
train_getter = getsentence(train_data)
train_sentences = train_getter.sentences

dev_getter = getsentence(dev_data)
dev_sentences = dev_getter.sentences

test_getter = getsentence_ForTest(test_data)
test_sentences = test_getter.sentences

#This is how a sentence will look like. 
print(train_sentences[0])

[('@SammieLynnsMom', 'NN', 'O'), ('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'inLocations': word.lower() in locations,
        'inNames' : word.lower() in names_list,
        'isTag' : word.find('@'),
        'hasCapitalLetter' : word[0].isupper(),
        # 'stem' : stemmer.stem(word),        
        'lemma' : lemmatizer.lemmatize(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            # '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:inLocations': word1.lower() in locations,
            '-1:inNames' : word1.lower() in names_list,
            '-1:hasCapitalLetter' : word1[0].isupper(),
            # '-1:stem' : stemmer.stem(word1),            
            '-1:lemma' : lemmatizer.lemmatize(word1),
             
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            # '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:inLocations': word1.lower() in locations,
            '+1:inNames' : word1.lower() in names_list,
            '+1:hasCapitalLetter' : word1[0].isupper(),
            '+1:stem' : stemmer.stem(word1),            
            '+1:lemma' : lemmatizer.lemmatize(word),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [55]:
print(test_sentences[5])
sent2features(train_sentences[3])[0]
# sent2features(test_sentences[7])[0]

[('Happy', 'JJ'), ('v-day', 'NN'), ('😌', 'NN'), ('❤', 'NN'), ('️', 'NN'), ('😂😂😂😂', 'NN'), ('#noshame', '#')]


{'+1:hasCapitalLetter': False,
 '+1:inLocations': False,
 '+1:inNames': False,
 '+1:lemma': '@ls_n',
 '+1:postag': 'RB',
 '+1:postag[:2]': 'RB',
 '+1:stem': 'perhap',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 'BOS': True,
 'bias': 1.0,
 'hasCapitalLetter': False,
 'inLocations': False,
 'inNames': False,
 'isTag': 0,
 'lemma': '@ls_n',
 'postag': 'NN',
 'postag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': '@ls_n',
 'word[-2:]': '_n',
 'word[-3:]': 's_n'}

In [0]:
#Creating the train and test set

X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]

X_test = [sent2features(s) for s in test_sentences]


In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
crf = crf.fit(X_train,y_train)

In [59]:
pred = cross_val_predict(estimator=crf, X = X_train,y = y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [60]:
#evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y_train)
print(report)
# f1 = metrics.flat_f1_score(y_train, y_pred, average='weighted', labels=labels)
# print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company       0.86      0.47      0.61       171
     B-group       0.62      0.17      0.27       106
  B-location       0.77      0.55      0.64       380
     B-other       0.53      0.25      0.34       225
    B-person       0.79      0.64      0.71       449
   B-product       0.80      0.21      0.33        97
     B-title       0.52      0.21      0.29        68
   I-company       0.50      0.11      0.18        36
     I-group       0.44      0.14      0.22        84
  I-location       0.60      0.35      0.44       154
     I-other       0.44      0.29      0.35       320
    I-person       0.77      0.72      0.74       215
   I-product       0.71      0.19      0.30        80
     I-title       0.44      0.19      0.27        77
           O       0.97      0.99      0.98     44007

    accuracy                           0.96     46469
   macro avg       0.65      0.37      0.44     46469
weighted avg       0.96   

In [62]:
# labels = list(crf.classes_)

# labels.remove('O')
print(labels)


['B-location', 'I-location', 'B-title', 'I-title', 'B-company', 'B-product', 'B-person', 'B-other', 'I-other', 'B-group', 'I-group', 'I-product', 'I-company', 'I-person']


In [63]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.3116337377240361

In [64]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("Accuracy: " + str(accuracy))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))

recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.667     0.154     0.250        39
   I-company      0.000     0.000     0.000        10
     B-group      0.600     0.027     0.052       111
     I-group      0.333     0.021     0.039        48
  B-location      0.627     0.448     0.523       154
  I-location      0.620     0.383     0.473        81
     B-other      0.489     0.174     0.257       132
     I-other      0.173     0.237     0.200        97
    B-person      0.619     0.532     0.572       171
    I-person      0.587     0.642     0.613        95
   B-product      1.000     0.027     0.053        37
   I-product      1.000     0.017     0.033       121
     B-title      0.000     0.000     0.000        17
     I-title      0.000     0.000     0.000        15

   micro avg      0.497     0.276     0.355      1128
   macro avg      0.480     0.190     0.219      1128
weighted avg      0.582     0.276     0.312      1128

Accuracy: 0.9412090277350

# Hyper Parameter Optimization

In [65]:
Model_X = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 27.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=False, averaging=None,
                                 c=None, c1=0.1, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_t...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7eff3af09f28>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighted, l

# Best Parameters

In [66]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.0039331454915196495, 'c2': 0.005457920820290508}
best CV score: 0.4697630895123395
model size: 1.30M


In [67]:
crf = rs.best_estimator_
y_pred = crf.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("accuracy :" + str(accuracy))
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))
recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.636     0.179     0.280        39
   I-company      0.000     0.000     0.000        10
     B-group      0.625     0.045     0.084       111
     I-group      0.250     0.021     0.038        48
  B-location      0.660     0.442     0.529       154
  I-location      0.667     0.395     0.496        81
     B-other      0.533     0.182     0.271       132
     I-other      0.209     0.237     0.222        97
    B-person      0.631     0.520     0.571       171
    I-person      0.589     0.589     0.589        95
   B-product      0.571     0.108     0.182        37
   I-product      0.333     0.017     0.031       121
     B-title      0.000     0.000     0.000        17
     I-title      0.000     0.000     0.000        15

   micro avg      0.519     0.276     0.360      1128
   macro avg      0.408     0.195     0.235      1128
weighted avg      0.512     0.276     0.324      1128

accuracy :0.9417010023983

In [68]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-person -> I-person 8.700147
B-title -> I-title 8.582140
B-other -> I-other 8.307213
B-location -> I-location 8.169060
I-title -> I-title 8.119032
B-product -> I-product 8.102421
B-group -> I-group 8.098150
I-group -> I-group 7.616215
I-product -> I-product 6.738798
I-other -> I-other 6.566214
B-company -> I-company 5.788404
I-location -> I-location 5.651181
I-person -> I-person 5.315123
I-company -> I-company 4.851213
O      -> O       4.228243
O      -> B-person 2.265310
O      -> B-title 2.046003
O      -> B-group 1.883943
O      -> B-product 1.605143
O      -> B-location 1.585559

Top unlikely transitions:
I-person -> O       -0.029275
B-location -> B-location -0.035175
I-title -> O       -0.056044
B-company -> O       -0.108513
B-other -> O       -0.132205
B-other -> B-person -0.135444
I-product -> B-product -0.288476
I-product -> O       -0.303473
B-product -> O       -0.330795
I-location -> B-person -0.367226
B-location -> O       -0.383697
I-company -> 

In [69]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.601599 B-company word.lower():twitter
5.716916 O        +1:stem:bless
5.517394 O        EOS
5.241207 B-location inLocations
5.241058 B-company word.lower():facebook
4.976367 O        bias
4.835718 B-person word.lower():pope
4.570714 B-product word.lower():ipod
4.349481 O        +1:stem:thi
4.197624 B-product +1:stem:devic
4.112117 B-location -1:lemma:at
4.109225 I-location -1:lemma:JANNUS
4.043498 I-other  -1:lemma:Guestmix
4.010778 B-person lemma:pope
4.010778 B-person +1:lemma:pope
3.995074 B-person word.lower():4dbling
3.995074 B-person lemma:4Dbling
3.995074 B-person +1:lemma:4Dbling
3.992067 B-company +1:stem:instant
3.968019 B-location +1:stem:wind
3.902225 B-company -1:lemma:@mckenziecomer
3.902225 B-company +1:stem:tube
3.866102 O        BOS
3.847311 B-group  +1:stem:gear
3.843243 B-person word.lower():billy
3.843243 B-person lemma:billy
3.843243 B-person -1:lemma:Silly
3.803043 B-person inNames
3.776927 I-other  -1:lemma:Bowl
3.736232 B-group  -1:lemma:Go

Top 

In [70]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(y_pred[0][0])

O


In [0]:
outR = open("test_no_tag.txt", "r")
outF = open("result.txt", "w")
lines = outR.readlines()
pred_line = 0
pred_word = 0
for line in lines:
  if(len(line)>1):
    outF.write(line.replace('\n','')+"\t\t"+y_pred[pred_line][pred_word]+"\n")
    pred_word = pred_word + 1
  else:
    outF.write("\n")
    pred_word = 0
    pred_line = pred_line + 1


outR.close()
outF.close()